# COMP5046 Assignment 1
*Make sure you change the file name with your unikey.*

# Readme
*If there is something to be noted for the user, please mention here.* 

*If you are planning to implement a program with Object Oriented Programming style*

***Visualising the comparison of different results is a good way to justify your decision.***

# 1 - Data Preprocessing (Personality chat datasets)

## 1.1. Download Dataset (Personality chat datasets)

In [1]:
# Please comment your code
# You should submit "ipynb" file (You can download it from "File" > "Download .ipynb") to Canvas
import json
import re

import tensorflow as tf
import numpy as np
import pandas as pd
import nltk

nltk.download('punkt')


# Code to download file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

id_comic = '11hwpf-M-lwYkVdsUX3xB97kHQ-jaAynM'
downloaded = drive.CreateFile({'id':id_comic}) 
downloaded.GetContentFile('qna_chitchat_the_comic.tsv')  

id_friend= '1IudiSdYUkmDB-RdLzKKkFRW0pAoa-q_w'
downloaded = drive.CreateFile({'id':id_friend}) 
downloaded.GetContentFile('qna_chitchat_the_friend.tsv')  

id_professional= '1bc4tr_wejEdmUfUGU7qlymMU_s1GhFgn'
downloaded = drive.CreateFile({'id':id_professional}) 
downloaded.GetContentFile('qna_chitchat_the_professional.tsv')  


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
    100% |████████████████████████████████| 993kB 20.8MB/s 


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
from gensim.parsing.preprocessing import remove_stopwords





df_professional= pd.read_csv('qna_chitchat_the_professional.tsv', sep="\t")
df_pro = df_professional.dropna()

df_friend = pd.read_csv('qna_chitchat_the_friend.tsv', sep="\t")
df_friend = df_friend.dropna()

df_comic = pd.read_csv('qna_chitchat_the_comic.tsv', sep="\t")
df_comic = df_comic.dropna()


/usr/local/lib/python3.6/dist-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


## 1.2. Preprocess data (Personality chat datasets)

 The preprocess technique that I used for preprocessing are 
 1. removing punctuations
 2. decapitalisation.
 
     The reason we removing punctuations is because we are using word2vec model, which transfering a word into a vector. Word2vec takes as its input a large corpus of text and produces a vector space. The two major methods are Continous Bag-Of-Words (CBOW) and Skip Gram. CBOW uses context word to estimate the center word of the sentence and vice versa for skip gram. 
     
     Punctuations are not important when we transfer it into vector since we are only guessing words, but not puncutations. Therefore, removing punctuations can reduce runtime effectively and prevent unnessary time for processing and trainning the word2vec model.
     
     When we tokenize our question from sentence to a single word. Decapitalisation is useful because we can ignore same word with different capital. For instance, a question with "Where" starting and "where" starting make no difference. However, without decapitalisation, "Where" and "where" will be tokenize as different token. Thus, runtime will be longer and by using this techinque, runtime will be minimize in a certain way. 
     
 3. Word-tokenize 
 
      There are two parts that we have been used word-tokneize technique. For questiones, we tokenize every word as a single token, for instance, "How are you" will be tokenize as 3 independent token ' How', ' are', 'you' . And for answers, we tokenize a signle repsonse as a token. For isntasnce, " i don t really have an age" will be tokenize as "i don t really have an age". 

In [3]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize
import re 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
# Please comment your code
def preProcess(file_name):
  unique_list =[]
  for index, row in file_name.iterrows():
    
    
    question = row['Question']
    question = re.sub(r'\([^)]*\)', '', question.lower())
    question = re.sub(r"[^a-z0-9]+", " ", question)
    tokenized_q = word_tokenize(question)
    unique_list.append(tokenized_q)
    
    answer = row['Answer']
    answer = re.sub(r'\([^)]*\)', '', answer.lower())
    answer = re.sub(r"[^a-z0-9]+", " ", answer).strip()
    unique_list.append([answer])
    
  unique_list.append(["_B_"])
  unique_list.append(["_E_"])
  unique_list.append(["_P_"])
  unique_list.append(["_U_"])
    

  return unique_list

pro_data = preProcess(df_pro)
friend_data = preProcess(df_friend)
comic_data = preProcess(df_comic)



In [5]:
print(len(pro_data))
print(len(friend_data))
print(len(comic_data))

1318
1318
1318


# 2 - Model Implementation

## 2.1. Word Embeddings

*You are required to describe which model was implemented (i.e. Word2Vec with CBOW, FastText with SkipGram, etc.) with justification of your decision *

In this assignment, I choose to use FastText with SkipGram. At first I would like to talk about why I choose FastText instead of Word2Vec. FastText is a library for learning of word embeddings and text classification created by Facebook’s AI Research (FAIR) lab. 

FastText is an extension to Word2Vec proposed by Facebook in 2016. Instead of feeding individual words into the Neural Network, FastText breaks words into several n-grams (sub-words). For instance, "apple" will be dervied to "app", "ppl" and "ple". Rare words are hard to detect or known by word2vec , but FastText by using the above method, can allow itself to know strange word because examples like "ppl" and "ple" may appear in other words. 

Moreover, SkipGram is easier to find rare words according to Mikolov. 



In [0]:
# Please comment your code
from gensim.models import FastText
from gensim.models import Word2Vec

### 2.1.1. Download Dataset for Word Embeddings



We have already imported FastText and Word2Vec model from gensim.models. As those models has already been pre-trained, therefore, we don't have to donwload the dataset for Word Embeddings. (Please refer to section 2.1 )

### 2.1.2. Data Preprocessing for Word Embeddings

Preprocess has already been done on section 1.2

### 2.1.3. Build Word Embeddings Model

Hyperparameters explaination:
1.  sorted_vocab  – This hyperparameters only take two input, which is 1 or 0. If we set this as 1, then we can get words that from descending order. By doing so, I can ensure my word-list after I load my s2s model in the same order, having the same order is quite important when we start chatting. 

 
2.   Because the max input length for s2s model is larger than 10, therefore, after I have tried different combination, I found that window size with 5 is the optimal number.

3. min_count =1 --  Since we have paddings like "_U_" and "_E_", in order to make sure those paddings are also inside our model, we have to set the min_count to 1. 




### 2.1.4. Train Word Embeddings Model

In [0]:
# Please comment your code
ft_sg_pro_model = FastText(sentences=pro_data, size=100, window=5, min_count=1, workers=4, sg=1, sorted_vocab=1)
ft_sg_com_model = FastText(sentences=comic_data, size=100, window=5, min_count=1, workers=4, sg=1, sorted_vocab=1)
ft_sg_friend_model = FastText(sentences=friend_data, size=100, window=5, min_count=1, workers=4, sg=1, sorted_vocab=1)

In [8]:
print(ft_sg_pro_model)
print(ft_sg_pro_model.wv.vocab)
print(len(ft_sg_pro_model.wv.vocab))


FastText(vocab=611, size=100, alpha=0.025)
{'what': <gensim.models.keyedvectors.Vocab object at 0x7fca23f19cc0>, 's': <gensim.models.keyedvectors.Vocab object at 0x7fca23f19c18>, 'your': <gensim.models.keyedvectors.Vocab object at 0x7fca23f19be0>, 'age': <gensim.models.keyedvectors.Vocab object at 0x7fca23f19d68>, 'age doesn t really apply to me': <gensim.models.keyedvectors.Vocab object at 0x7fca23f19cf8>, 'are': <gensim.models.keyedvectors.Vocab object at 0x7fca23f19b38>, 'you': <gensim.models.keyedvectors.Vocab object at 0x7fca23f19c50>, 'young': <gensim.models.keyedvectors.Vocab object at 0x7fca22241dd8>, 'when': <gensim.models.keyedvectors.Vocab object at 0x7fca2218cc50>, 'were': <gensim.models.keyedvectors.Vocab object at 0x7fca22275e80>, 'born': <gensim.models.keyedvectors.Vocab object at 0x7fca223fcc18>, 'old': <gensim.models.keyedvectors.Vocab object at 0x7fca223fc048>, 'how': <gensim.models.keyedvectors.Vocab object at 0x7fca223fc080>, 'long': <gensim.models.keyedvectors.Voca

In [9]:
print(pro_data)

[['what', 's', 'your', 'age'], ['age doesn t really apply to me'], ['are', 'you', 'young'], ['age doesn t really apply to me'], ['when', 'were', 'you', 'born'], ['age doesn t really apply to me'], ['what', 'age', 'are', 'you'], ['age doesn t really apply to me'], ['are', 'you', 'old'], ['age doesn t really apply to me'], ['how', 'old', 'are', 'you'], ['age doesn t really apply to me'], ['how', 'long', 'ago', 'were', 'you', 'born'], ['age doesn t really apply to me'], ['ask', 'me', 'anything'], ['i m better at answering questions'], ['ask', 'me', 'a', 'question'], ['i m better at answering questions'], ['can', 'you', 'ask', 'me', 'a', 'question'], ['i m better at answering questions'], ['ask', 'me', 'something'], ['i m better at answering questions'], ['what', 'do', 'you', 'want', 'to', 'know', 'about', 'me'], ['i m better at answering questions'], ['can', 'you', 'sleep'], ['i don t have a body'], ['do', 'you', 'have', 'boogers'], ['i don t have a body'], ['don', 't', 'you', 'ever', 'sl

In [10]:
print(friend_data)

[['what', 's', 'your', 'age'], ['i don t really have an age'], ['are', 'you', 'young'], ['i don t really have an age'], ['when', 'were', 'you', 'born'], ['i don t really have an age'], ['what', 'age', 'are', 'you'], ['i don t really have an age'], ['are', 'you', 'old'], ['i don t really have an age'], ['how', 'old', 'are', 'you'], ['i don t really have an age'], ['how', 'long', 'ago', 'were', 'you', 'born'], ['i don t really have an age'], ['ask', 'me', 'anything'], ['i m a much better answerer than asker'], ['ask', 'me', 'a', 'question'], ['i m a much better answerer than asker'], ['can', 'you', 'ask', 'me', 'a', 'question'], ['i m a much better answerer than asker'], ['ask', 'me', 'something'], ['i m a much better answerer than asker'], ['what', 'do', 'you', 'want', 'to', 'know', 'about', 'me'], ['i m a much better answerer than asker'], ['can', 'you', 'sleep'], ['i don t have the hardware for that'], ['do', 'you', 'have', 'boogers'], ['i don t have the hardware for that'], ['don', '

### 2.1.5. Save Word Embeddings Model

In [0]:
# Please comment your code
ft_sg_pro_model.save("ft_sg_proo.model")
ft_sg_com_model.save("ft_sg_comm.model")
ft_sg_friend_model.save("ft_sg_friendd.model")

### 2.1.6. Load Word Embeddings Model

In [0]:
# Please comment your code
ft_sg_pro_model = FastText.load("ft_sg_proo.model")
ft_sg_com_model = FastText.load("ft_sg_comm.model")
ft_sg_friend_model = FastText.load("ft_sg_friendd.model")


from gensim.models import Word2Vec
from gensim.models import FastText

# what are those id here ? 
# https://drive.google.com/file/d/1s8CtIIGTYa-SPJuvn8QjgsmSK4GNTxtT/view?usp=sharing

id_0 = '1s8CtIIGTYa-SPJuvn8QjgsmSK4GNTxtT'
# https://drive.google.com/file/d/1puf3fqUV4CJQ3dHq1Ydavllzjf9wwW3h/view?usp=sharing

id_1 = '1puf3fqUV4CJQ3dHq1Ydavllzjf9wwW3h'

# https://drive.google.com/file/d/1Yt6X-YGW1N-_qVmtV1yi7vtN0J7BmkoN/view?usp=sharing
id_2 = '1Yt6X-YGW1N-_qVmtV1yi7vtN0J7BmkoN'

downloaded_pro = drive.CreateFile({'id':id_0}) 
downloaded_pro.GetContentFile('ft_sg_proo.model')  

downloaded_friend = drive.CreateFile({'id':id_1}) 
downloaded_friend.GetContentFile('ft_sg_friendd.model')  

downloaded_comic = drive.CreateFile({'id':id_2}) 
downloaded_comic.GetContentFile('ft_sg_comm.model') 


pro_model = FastText.load("ft_sg_proo.model")
pro_len = len(pro_model.wv.vocab)
friend_model = FastText.load("ft_sg_friendd.model")
friend_len = len(friend_model.wv.vocab)
comic_model = FastText.load("ft_sg_comm.model")
comic_len = len(comic_model.wv.vocab)


## 2.2. Seq2Seq model

### 2.2.1. Apply/Import Word Embedding Model

*You are required to describe how hyperparameters were decided with justification of your decision.*

Hyperparameters explainations:

max_input_words_amount: Because we are going to build a N to one s2s model, this parameters should take the maximum length of the model into considertaions. Therefore, this parameters may vary. In the case of pro_model, it would be 12 (refer to pro_max_input below. ).

dic_len:  Dic_len is very important in our model since one hot vector is needed in s2s model. Therefore, the length of the one hot vector would be "FastText.wv.vocab".

In [0]:
# Please comment your code

### 2.2.2. Build Seq2Seq Model

*You are required to describe how hyperparameters were decided with justification of your decision.*

In [0]:
# Please comment your code
# Sequence data
# Generate unique tokens list from qas.json
# Please comment your code

def get_seq_process(name):
  seq_data = []
  whole_words = []
  max_input_words_amount = 0
  max_output_words_amount = 1


  for index, row in name.iterrows():
#     preprocess needed for both question and answer 
    question = row['Question']
    answer = row['Answer']
    
    question = re.sub(r'\([^)]*\)', '', question.lower())
    question = re.sub(r"[^a-z0-9]+", " ", question).strip()
    

    answer = re.sub(r'\([^)]*\)', '', answer.lower())
    answer = re.sub(r"[^a-z0-9]+", " ", answer).strip()
    
    seq_data.append([question,answer])
    tokenized_q = word_tokenize(question)
    tokenized_a = [answer]
    
    max_input_words_amount = max(len(tokenized_q), max_input_words_amount)
    max_output_words_amount = max(len(tokenized_a), max_output_words_amount)
    
    
    
    
  return  max_input_words_amount ,seq_data



In [0]:
pro_max_input, pro_seq_data = get_seq_process(df_pro)
pro_length = len(ft_sg_pro_model.wv.vocab)

friend_max_input,friend_seq_data = get_seq_process(df_friend)
friend_length = len(ft_sg_friend_model.wv.vocab)

comic_max_input, comic_seq_data = get_seq_process(df_comic)
comic_length = len(ft_sg_com_model.wv.vocab)

In [16]:
print(pro_seq_data, pro_length)

[['what s your age', 'age doesn t really apply to me'], ['are you young', 'age doesn t really apply to me'], ['when were you born', 'age doesn t really apply to me'], ['what age are you', 'age doesn t really apply to me'], ['are you old', 'age doesn t really apply to me'], ['how old are you', 'age doesn t really apply to me'], ['how long ago were you born', 'age doesn t really apply to me'], ['ask me anything', 'i m better at answering questions'], ['ask me a question', 'i m better at answering questions'], ['can you ask me a question', 'i m better at answering questions'], ['ask me something', 'i m better at answering questions'], ['what do you want to know about me', 'i m better at answering questions'], ['can you sleep', 'i don t have a body'], ['do you have boogers', 'i don t have a body'], ['don t you ever sleep', 'i don t have a body'], ['do you dream', 'i don t have a body'], ['do you smell', 'i don t have a body'], ['do you sweat', 'i don t have a body'], ['do you get tired', '

In [17]:
print(pro_max_input)

12


In [0]:
# Please comment your code
# get token index vector of questions and add paddings if the word is shorter than the maximum number of words
def get_vectors_q(sentence, max_input, model):
    
    temp_sentence = re.sub(r'\([^)]*\)', '', sentence)
    tokenized_sentence = word_tokenize(temp_sentence)
    
    diff = max_input - len(tokenized_sentence)
    
    for x in range(diff):
        tokenized_sentence.append(  "_P_"        )
        
    data=[]
    for token in tokenized_sentence:
        if token in model.wv.vocab:
            data.append(    model[token]  )
        else:
            data.append(   model['_U_']   )
    
    return data

# get token index vector of answer
def get_vectors_a(sentence,model):    
    tokenized_sentence = [sentence]
    data = tokens_to_ids(tokenized_sentence,model)
    
    return data
    
    

# convert tokens to index
def tokens_to_ids(tokenized_sentence, model):
    ids = []

    for token in tokenized_sentence:
        if token in num_dic:
            ids.append(     model[token]          )
        else:
            ids.append(     model["_U_"]          )

    return ids


# generate a batch data for training/testing
def make_batch(seq_data, max_input, model, dic_len):
    input_batch = []
    output_batch = []
    target_batch = []
    count = 0

    for seq in seq_data:        

        wordVocab = []
        for k in model.wv.vocab:
          wordVocab.append(k)
          
        input_sequence = []
        tokenized_sentence = word_tokenize(seq[0])
        diff = max_input - len(tokenized_sentence)
        for x in range(diff):
            tokenized_sentence.append(['_P_'])
            
        for i in tokenized_sentence:
          if i in wordVocab:
            temp = wordVocab.index(i)
          else:
            temp = wordVocab.index('_U_')
          input_sequence.append(temp)
#         print(len(input_sequence))
#         print(input_sequence,count)
        output_sequence = []
        output_sequence.append(wordVocab.index('_B_'))
#         for i in word_tokenize(seq[1]):
        if seq[1] in wordVocab:
          temp = wordVocab.index(seq[1])
        else:
          temp = wordVocab.index('_U_')
        output_sequence.append(temp)
#         print(output_sequence,count)
        input_batch.append(np.eye(dic_len)[input_sequence])
        output_batch.append(np.eye(dic_len)[output_sequence])
#         print(len(output_batch[0][0]),output_batch)
        target = []
        
        if seq[1] in wordVocab:
          temp_tar = wordVocab.index(seq[1])
          target.append(temp_tar)
        else:
          temp_tar = wordVocab.index('_U_')
          target.append(temp_tar)
        target.append(wordVocab.index('_E_'))
        
        target_batch.append(target)
#         print(target, count)
        count = count +1
#     print(target_batch)
    return input_batch, output_batch, target_batch



In [0]:
# Generate batch data

pro_input_batch, pro_output_batch,  pro_target_batch    = make_batch(pro_seq_data, pro_max_input, pro_model, pro_length) 
friend_input_batch, friend_output_batch, friend_target_batch = make_batch(friend_seq_data, friend_max_input, friend_model, friend_length)
comic_input_batch,  comic_output_batch, comic_target_batch  = make_batch(comic_seq_data, comic_max_input, comic_model, comic_length)

In [0]:
### Setting Hyperparameters
def S2Strain(total_epoch, model_len, input_batch, output_batch, target_batch, name): 
  learning_rate = 0.002
  n_hidden = 128

  n_class = model_len
  n_input = model_len

### Neural Network Model
  tf.reset_default_graph()

# encoder/decoder shape = [batch size, time steps, input size]
  enc_input = tf.placeholder(tf.float32, [None, None, n_input])
  dec_input = tf.placeholder(tf.float32, [None, None, n_input])

# target shape = [batch size, time steps]
  targets = tf.placeholder(tf.int64, [None, None])


# Encoder Cell
  with tf.variable_scope('encode'):
    enc_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob=0.5)

    outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input,
                                            dtype=tf.float32)
# Decoder Cell
  with tf.variable_scope('decode'):
    dec_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.5)

    # [IMPORTANT] Setting enc_states as inital_state of decoder cell
    outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, dec_input,
                                            initial_state=enc_states,
                                            dtype=tf.float32)

  model = tf.layers.dense(outputs, n_class, activation=None)

  cost = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits=model, labels=targets))

  optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

    
### Training Model
# sess = tf.Session()
# sess.run(tf.global_variables_initializer())


  saver = tf.train.Saver()
  sess = tf.Session()
#   with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  total_epoch = total_epoch

  for epoch in range(total_epoch):
        _, loss = sess.run([optimizer, cost],
                           feed_dict={enc_input: input_batch,
                                      dec_input: output_batch,
                                      targets: target_batch})
        if epoch % 100 == 0:
          print('Epoch:', '%04d' % (epoch + 1),
                'cost =', '{:.6f}'.format(loss))

  print('Epoch:', '%04d' % (epoch + 1),
          'cost =', '{:.6f}'.format(loss))
  print('Training completed')
  save_path = saver.save(sess, name)
  print("Model saved in path: %s" % save_path)
  print(model)
  return model, sess, saver


# Generate a batch data
# pro_dic_len, pro_max_input, pro_max_output, pro_num_dic, pro_seq_data
# def make_batch(seq_data,max_input,max_output,num_dic,dic_len):

In [21]:
pro_s2s_model, pro_sess, pro_saver= S2Strain(5000, pro_length, pro_input_batch, pro_output_batch, pro_target_batch,"pro_s2s.ckpt")

Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.dense instead.
Epoch: 0001 cost = 6.402759
Epoch: 0101 cost = 2.265990
Epoch: 0201 cost = 2.253886
Epoch: 0301 cost = 2.243835
Epoch: 0401 cost = 2.243989
Epoch: 0501 cost = 2.238602
Epoch: 0601 cost = 2.233913
Epoch: 0701 cost = 2.239787
Epoch: 0801 cost = 2.242383
Epoch: 0901 cost = 2.239371
Epoch: 1001 cost = 2.234358
Epoch: 1101 cost = 1.852648
Epoch: 1201 cost = 1.238141
Epoch: 1301 cost = 0.992493
Epoch: 1401 cost = 0.892945
Epoch: 1501 cost = 0.769707
Epoch: 1601 cost = 0.698667
Epoch: 1701 cost = 0.649300
Epoch: 1801 cost = 0

In [22]:
friend_s2s_model, friend_sess, friend_saver= S2Strain(5000, friend_length, friend_input_batch, friend_output_batch, friend_target_batch,"friend_s2s.ckpt")

Epoch: 0001 cost = 6.410337
Epoch: 0101 cost = 2.259082
Epoch: 0201 cost = 2.238534
Epoch: 0301 cost = 2.236269
Epoch: 0401 cost = 2.225468
Epoch: 0501 cost = 2.227412
Epoch: 0601 cost = 2.227373
Epoch: 0701 cost = 2.228634
Epoch: 0801 cost = 2.236188
Epoch: 0901 cost = 2.230300
Epoch: 1001 cost = 2.223103
Epoch: 1101 cost = 2.226608
Epoch: 1201 cost = 2.220703
Epoch: 1301 cost = 2.229931
Epoch: 1401 cost = 2.225750
Epoch: 1501 cost = 2.224352
Epoch: 1601 cost = 2.218582
Epoch: 1701 cost = 1.775199
Epoch: 1801 cost = 0.943607
Epoch: 1901 cost = 0.652816
Epoch: 2001 cost = 0.410945
Epoch: 2101 cost = 0.300779
Epoch: 2201 cost = 0.305120
Epoch: 2301 cost = 0.205878
Epoch: 2401 cost = 0.170026
Epoch: 2501 cost = 0.154768
Epoch: 2601 cost = 0.140738
Epoch: 2701 cost = 0.110421
Epoch: 2801 cost = 0.108270
Epoch: 2901 cost = 0.354027
Epoch: 3001 cost = 0.109014
Epoch: 3101 cost = 0.090901
Epoch: 3201 cost = 0.086118
Epoch: 3301 cost = 0.088041
Epoch: 3401 cost = 0.072064
Epoch: 3501 cost = 0

In [23]:
comic_s2s_model, comic_sess, comic_saver= S2Strain(5000, comic_length, comic_input_batch, comic_output_batch, comic_target_batch,"comic_s2s.ckpt")

Epoch: 0001 cost = 6.410897
Epoch: 0101 cost = 2.265137
Epoch: 0201 cost = 2.252103
Epoch: 0301 cost = 2.246473
Epoch: 0401 cost = 2.243267
Epoch: 0501 cost = 2.244953
Epoch: 0601 cost = 2.244730
Epoch: 0701 cost = 2.237655
Epoch: 0801 cost = 2.235316
Epoch: 0901 cost = 1.696796
Epoch: 1001 cost = 1.268118
Epoch: 1101 cost = 1.012852
Epoch: 1201 cost = 0.853111
Epoch: 1301 cost = 0.699515
Epoch: 1401 cost = 0.637753
Epoch: 1501 cost = 0.590885
Epoch: 1601 cost = 0.454768
Epoch: 1701 cost = 0.417869
Epoch: 1801 cost = 0.483163
Epoch: 1901 cost = 0.333514
Epoch: 2001 cost = 0.308573
Epoch: 2101 cost = 0.313912
Epoch: 2201 cost = 0.266706
Epoch: 2301 cost = 0.252153
Epoch: 2401 cost = 0.277371
Epoch: 2501 cost = 0.241165
Epoch: 2601 cost = 0.230183
Epoch: 2701 cost = 0.407007
Epoch: 2801 cost = 0.201704
Epoch: 2901 cost = 0.196588
Epoch: 3001 cost = 0.207164
Epoch: 3101 cost = 0.186131
Epoch: 3201 cost = 0.633854
Epoch: 3301 cost = 0.175623
Epoch: 3401 cost = 0.165209
Epoch: 3501 cost = 0

### 2.2.3. Train Seq2Seq Model

### 2.2.4. Save Seq2Seq Model

In [0]:
# Please comment your code


### 2.2.5. Load Seq2Seq Model

In [0]:
# Please comment your code
id_00 = '1mi_HbezZuBEpretXih-gOTzzd66ZAHm2'
id_01 = '1SpNun9vYOzO6BvX0UMy6P7eCyX-6cApD'
id_02 = '1h4JX2OD7nH0OQvplwNQ-P1TLoAuCXWqg'

downloaded_pro = drive.CreateFile({'id':id_00}) 
downloaded_pro.GetContentFile('pro_s2s.ckpt.data-00000-of-00001')  

downloaded_friend = drive.CreateFile({'id':id_01}) 
downloaded_friend.GetContentFile('pro_s2s.ckpt.index')  

downloaded_comic = drive.CreateFile({'id':id_02}) 
downloaded_comic.GetContentFile('pro_s2s.ckpt.meta')  

In [0]:
id_0 = '1ABR54m4kr8wH2v_YxcVWcuSJZYPovNrs'

id_1 = '135irYQsvVkB06FcuU7qtj25ei6pI17o-'


id_2 = '1jgZgOxAqqvDAo8jmII0Sgd5jEu4rxxRj'
downloaded = drive.CreateFile({'id':id_0}) 
downloaded.GetContentFile('friend_s2s.ckpt.data-00000-of-00001')  

downloaded_friend = drive.CreateFile({'id':id_1}) 
downloaded_friend.GetContentFile('friend_s2s.ckpt.index')  

downloaded_comic = drive.CreateFile({'id':id_2}) 
downloaded_comic.GetContentFile('friend_s2s.ckpt.meta')  

In [0]:
id_0 = '1B3E1U94WPgi4gqR3GpurmKTNTrfF8nji'


id_1 = '1hZHKgsMYDy6kz9JkhrTWkUjGnQ7EazWQ'


id_2 = '19hR6Hcsv2at3r3_9GJyfMV1lc1LYK0Y4'


downloaded = drive.CreateFile({'id':id_0}) 
downloaded.GetContentFile('comic_s2s.ckpt.data-00000-of-00001')  

downloaded_friend = drive.CreateFile({'id':id_1}) 
downloaded_friend.GetContentFile('comic_s2s.ckpt.index')  

downloaded_comic = drive.CreateFile({'id':id_2}) 
downloaded_comic.GetContentFile('comic_s2s.ckpt.meta')  

# 3 - Evaluation (Running chatbot)

## 3.1. Start chatting

In [0]:
import sys
import logging

In [0]:
def ask_pro(question):
  tf.reset_default_graph()
  with tf.Session() as sess:

      sess.run(tf.global_variables_initializer())

      learning_rate = 0.002
      n_hidden = 128
      n_class = pro_len
      n_input = pro_len
      enc_input = tf.placeholder(tf.float32, [None, None, n_input])
      dec_input = tf.placeholder(tf.float32, [None, None, n_input])
      targets = tf.placeholder(tf.int64, [None, None])
      with tf.variable_scope('encode'):
        enc_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
        enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob=0.5)
        outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input,
                                              dtype=tf.float32)
        
      with tf.variable_scope('decode'):
        dec_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
        dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.5)

        outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, dec_input,
                                            initial_state=enc_states,
                                            dtype=tf.float32)
      model = tf.layers.dense(outputs, n_class, activation=None)

      cost = tf.reduce_mean(
                tf.nn.sparse_softmax_cross_entropy_with_logits(
                    logits=model, labels=targets))

      optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
    
      pro_saver = tf.train.Saver()
      pro_saver.restore(sess, "./pro_s2s.ckpt")
#       print("Pro_Model restored.")
    
      max_output_words_amount=1
#     sentence = "Can you sleep?"
      sentence = question
      sentence = re.sub(r'\([^)]*\)', '', sentence.lower())
      sentence = re.sub(r"[^a-z0-9]+", " ", sentence)
      seq_data = [sentence, '_U_' * max_output_words_amount]
    
    
      input_batch, output_batch, target_batch = make_batch([seq_data],pro_max_input,pro_model, pro_len)
    
    
      prediction = tf.argmax(model, 2)

      result = sess.run(prediction,
                        feed_dict={enc_input:input_batch,
                                   dec_input:output_batch,
                                   targets:target_batch})
      unique_words = []
      for k in pro_model.wv.vocab:
        unique_words.append(k)
      decoded = [unique_words[i] for i in result[0]]
        
      if "_E_" in decoded:
        end = decoded.index('_E_')
        translated = ' '.join(decoded[:end])
      else :
        translated = ' '.join(decoded[:])
      
    
  return translated
   

In [0]:
def ask_friend(question):
  tf.reset_default_graph()
  with tf.Session() as sess:

      sess.run(tf.global_variables_initializer())

      learning_rate = 0.002
      n_hidden = 128
      n_class = friend_len
      n_input = friend_len
      enc_input = tf.placeholder(tf.float32, [None, None, n_input])
      dec_input = tf.placeholder(tf.float32, [None, None, n_input])
      targets = tf.placeholder(tf.int64, [None, None])
      with tf.variable_scope('encode'):
        enc_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
        enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob=0.5)
        outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input,
                                              dtype=tf.float32)
        
      with tf.variable_scope('decode'):
        dec_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
        dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.5)

        outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, dec_input,
                                            initial_state=enc_states,
                                            dtype=tf.float32)
      model = tf.layers.dense(outputs, n_class, activation=None)

      cost = tf.reduce_mean(
                tf.nn.sparse_softmax_cross_entropy_with_logits(
                    logits=model, labels=targets))

      optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
    
      friend_saver = tf.train.Saver()
      friend_saver.restore(sess, "./friend_s2s.ckpt")
#       print("Friend_Model restored.")
    
      max_output_words_amount=1
#     sentence = "Can you sleep?"
      sentence = question
      sentence = re.sub(r'\([^)]*\)', '', sentence.lower())
      sentence = re.sub(r"[^a-z0-9]+", " ", sentence)
      seq_data = [sentence, '_U_' * max_output_words_amount]
    
    
      input_batch, output_batch, target_batch = make_batch([seq_data],friend_max_input,friend_model, friend_len)
    
    
      prediction = tf.argmax(model, 2)

      result = sess.run(prediction,
                        feed_dict={enc_input:input_batch,
                                   dec_input:output_batch,
                                   targets:target_batch})
      unique_words = []
      for k in pro_model.wv.vocab:
        unique_words.append(k)
      decoded = [unique_words[i] for i in result[0]]
        
      if "_E_" in decoded:
        end = decoded.index('_E_')
        translated = ' '.join(decoded[:end])
      else :
        translated = ' '.join(decoded[:1])
      
    
  return translated
    

In [0]:
def ask_comic(question):
  tf.reset_default_graph()
  with tf.Session() as sess:

      sess.run(tf.global_variables_initializer())

      learning_rate = 0.002
      n_hidden = 128
      n_class = comic_len
      n_input = comic_len
      enc_input = tf.placeholder(tf.float32, [None, None, n_input])
      dec_input = tf.placeholder(tf.float32, [None, None, n_input])
      targets = tf.placeholder(tf.int64, [None, None])
      with tf.variable_scope('encode'):
        enc_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
        enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob=0.5)
        outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input,
                                              dtype=tf.float32)
        
      with tf.variable_scope('decode'):
        dec_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
        dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.5)

        outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, dec_input,
                                            initial_state=enc_states,
                                            dtype=tf.float32)
      model = tf.layers.dense(outputs, n_class, activation=None)

      cost = tf.reduce_mean(
                tf.nn.sparse_softmax_cross_entropy_with_logits(
                    logits=model, labels=targets))

      optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
    
      comic_saver = tf.train.Saver()
      comic_saver.restore(sess, "./comic_s2s.ckpt")
#       print("Friend_Model restored.")
    
      max_output_words_amount=1
#     sentence = "Can you sleep?"
      sentence = question
      sentence = re.sub(r'\([^)]*\)', '', sentence.lower())
      sentence = re.sub(r"[^a-z0-9]+", " ", sentence)
      seq_data = [sentence, '_U_' * max_output_words_amount]
    
    
      input_batch, output_batch, target_batch = make_batch([seq_data],comic_max_input,comic_model, comic_len)
    
    
      prediction = tf.argmax(model, 2)

      result = sess.run(prediction,
                        feed_dict={enc_input:input_batch,
                                   dec_input:output_batch,
                                   targets:target_batch})
      unique_words = []
      for k in pro_model.wv.vocab:
        unique_words.append(k)
      decoded = [unique_words[i] for i in result[0]]
        
      if "_E_" in decoded:
        end = decoded.index('_E_')
        translated = ' '.join(decoded[:end])
      else :
        translated = ' '.join(decoded[:])
      
    
  return translated

## 3.2. Change Personality

*Explain how to change personality (What is the command for changing personality?). *

In [0]:
# Please comment your code

## 3.3. Save chat log

Please run section 3.5.2 Execute chatting before you run section 3.3 Save chat log, because document 'mleu5455_chat_log.txt' has not been created in this section, in order to download it, we have to use the function 'file = open(mleu5455_chat_log.txt)' to create the file first, after you done talking with the chatbot, then you can run this part to download the file.

In [0]:

from google.colab import files
files.download('mleu5455_chat_log.txt')

## 3.4. End chatting

In [0]:
# Please comment your code

## 3.5. Execute program

***Please make sure your program  is running properly.***

***Functions for downloading (from Google Drive) and loading models (both word embeddings and Seq2Seq) need to be called!*** 


### 3.5.1. Execute program - training mode

Model has been trained from the above section. 

### 3.5.2. Execute program - chatting mode

*Please include lines to start chatting with the bot.*

Key sentences to change different model : 

talk as a profession ===> will activate profession model

talk as a friend ====> will activate friend model

talk as a comic guy ===> will activate comic model

finish chatting ====> to terminate the chatbot

end conversation ===> to terminate the chatbot

no more talk ======> to terminate the chatbot

In [33]:
# Please comment your code
# Please comment your code
import sys

file = open('mleu5455_chat_log.txt','w') 


end_chat ={"finish chatting":True, "end conversation":True, "no more talk": True}
change_model = {"talk as a profession":"pro_model",
                "talk as a friend":"friend_model",
                "talk as a comic guy":"comic_model"
               }
continue_chat = True
print("Hi, welcome to chatbot, how are you today? ")
model_chat = 'pro_model'
while continue_chat:
  model_changed = False
  sys.stdout.write ( 'Me: ')
  chat_input =input().lower()
  file.write('Me: '+ chat_input[0].upper() + chat_input[1:] + '.\n')
  for k in end_chat:
    if chat_input == k:
      file.write('ChatBot----->It\'s so nice to meet you, see you next time. Goodbye')
      sys.stdout.write('ChatBot----->It\'s so nice to meet you, see you next time. Goodbye')
      continue_chat = False

  if continue_chat:
    if chat_input in change_model:
        print(change_model[chat_input],' activated')
        file.write('\n---------------------'+ change_model[chat_input].upper() + 'ACTIVATED-----------------\n')
        model_chat = change_model[chat_input]
        model_changed = True
    
  if continue_chat and model_changed == False:
    answer = ''
    if model_chat == 'pro_model':
      answer = ask_pro(chat_input)
    elif model_chat == 'friend_model':
      answer = ask_friend(chat_input)
    elif model_chat == 'comic_model':
      answer = ask_comic(chat_input)
    
    char = answer[0].upper()
    answer =  'ChatBot----->'+ char + answer[ 1:] + '.\n'
    file.write(answer)
    sys.stdout.write(answer)
    
file.close()
#     sys.stdout.write("Machine Response for ---chat_input---\n")




Hi, welcome to chatbot, how are you today? 
Me: Good how are you today?
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./pro_s2s.ckpt
ChatBot----->That s great.
Me: I love you, do you know it?
INFO:tensorflow:Restoring parameters from ./pro_s2s.ckpt
ChatBot----->I aim to serve.
Me: What do you mean
INFO:tensorflow:Restoring parameters from ./pro_s2s.ckpt
ChatBot----->I think i might have gotten lost there.
Me: You are stupid 
INFO:tensorflow:Restoring parameters from ./pro_s2s.ckpt
ChatBot----->Ok.
Me: Are you angry?
INFO:tensorflow:Restoring parameters from ./pro_s2s.ckpt
ChatBot----->I try but i don t always get it right.
Me: What?
INFO:tensorflow:Restoring parameters from ./pro_s2s.ckpt
ChatBot----->Let s keep things professional.
Me: I hate you
INFO:tensorflow:Restoring parameters from ./pro_s2s.ckpt
ChatBot----->I try but i don t always get it right.
Me: talk as a friend
friend_model  activated
Me: H

## Object Oriented Programming codes here

*If you have multiple classes use multiple code snippets to add them.*

In [0]:
# If you used OOP style, use this sectioon